In [1]:
import pandas as pd
import json
import ast
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
pd.options.display.max_columns = None

In [5]:
df3 = pd.read_csv('data_processed_add_stems.csv', encoding = 'utf-16')

In [7]:
df3.columns

Index(['Unnamed: 0', 'id', 'name', 'salary.from', 'salary.to', 'alternate_url',
       'salary.currency', 'salary.gross', 'experience.name', 'experience.id',
       'schedule.name', 'employment.name', 'employer.name', 'requirement',
       'skills', 'description', 'prof_role', 'specializations',
       'description_stem'],
      dtype='object')

In [9]:
df2 = df3[['id', 'name', 'salary.from', 'salary.to', 'alternate_url',
       'salary.currency', 'salary.gross', 'experience.name', 'experience.id', 'schedule.name',
       'employment.name', 'employer.name', 'requirement', 'description', 'prof_role', 'specializations',
       'description_stem']]

In [10]:
df3.head(3)

,Unnamed: 0,id,name,salary.from,salary.to,alternate_url,salary.currency,salary.gross,experience.name,experience.id,schedule.name,employment.name,employer.name,requirement,skills,description,prof_role,specializations,description_stem
0,0,67716839,Специалист по работе с покупателями,50000.0,NaN,https://hh.ru/vacancy/67716839,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,ВкусВилл,ПК или ноутбук. Устойчивый интернет.,"Пользователь ПК,Грамотная речь,Поиск информаци...",Наша поддержка с душой реагирует на каждое обр...,Специалист технической поддержки,"Информационные технологии, интернет, телеком",наш поддержк с душ реагир на кажд обращен и вс...
1,1,68539555,Frontend разработчик (junior/middle),60000.0,120000.0,https://hh.ru/vacancy/68539555,RUR,False,От 1 года до 3 лет,between1And3,Полный день,Полная занятость,Инфоурок,"HTML(5)/CSS(3), Flex, кроссбраузерность. Препр...","JavaScript,Git,jQuery,HTML5,Sass",О нас: Продуктовая компания (основной продукт...,"Программист, разработчик","Информационные технологии, интернет, телеком",о нас продуктов компан основн продукт сайт кру...
2,2,68731055,Корректор контента,80000.0,NaN,https://hh.ru/vacancy/68731055,RUR,True,От 1 года до 3 лет,between1And3,Удаленная работа,Полная занятость,Российская академия народного хозяйства и госу...,"Знание Microsoft Office (PowerPoint, Word, Exc...","Грамотность,Грамотная речь,Работа в команде,Де...",В команду по развитию онлайн-образования требу...,"Копирайтер, редактор, корректор","Маркетинг, реклама, PR;Информационные технолог...",в команд по развит онлайн образован треб специ...


In [11]:
resumes2 = pd.read_csv('resumes_step2_add_stems.csv', encoding = 'utf-16')

In [12]:
resumes2.head(3)

,Unnamed: 0,birth_date,gender,area,education,title,education_level,salary.amount,salary.currency,specialization,skill_set,skills_desc,exp_length,exp_description,skills_desc_stem,exp_desc_stem
0,0,27 октября 1954,Мужчина,Москва,"[{'year': 1976, 'name': 'Высшее Военное Инжене...","Организатор корпоративных охот, рыбалок и эко-...",Высшее образование,60000.0,руб.,Руководитель проектов,"Работа в Фондах, НКО и ТПП, Союзах и Ассоциаци...",В работе: Аналитик.Трудоголик. Хобби: 50 лет ...,10915.0,Руководство административно-хозяйственной деят...,в работ аналитик трудоголик хобб лет занима ох...,руководств административн хозяйствен деятельн ...
1,1,7 июля 1978,Мужчина,Москва,"[{'year': 2002, 'name': 'Таганрогский Радиотех...",Руководитель проектов,Высшее образование,NaN,NaN,Руководитель проектов,Работоспособность,"Являюсь разносторонним человеком, люблю хорошо...",7554.0,"Организация рабочего процесса на объекте , вза...",явля разносторон человек любл хорош и плодотво...,организац рабоч процесс на объект взаимодейств...
2,2,20 января 1986,Мужчина,Москва,"[{'year': 2012, 'name': 'Институт Проблем Риск...",Руководитель проекта,Высшее образование,NaN,NaN,Руководитель проектов,"Автоматизация процессов,Проектное управление,О...","Системное мышление, глубокая аналитика и деком...",3732.0,Формирование бизнес требований. Сбор информац...,системн мышлен глубок аналитик и декомпозиц пр...,формирован бизнес требован сбор информац описа...


In [14]:
df3['description_stem'] = df3['description_stem'].fillna(' ')

# Схожесть требований и опыта

In [55]:
%time

matches = pd.DataFrame()
cv=CountVectorizer()

for i in range(len(df3)):
    
    resume = resumes2.loc[1, 'exp_desc_stem']
    vacancy = df3.loc[i, 'description_stem']
    
    Match_Test=[resume, vacancy]
    
    count_matrix=cv.fit_transform(Match_Test)
    
    MatchPercentage=cosine_similarity(count_matrix)[0][1]*100
    MatchPercentage=round(MatchPercentage,2)
    
    matches.loc[i, 'Similarity'] = round(MatchPercentage,2)
    matches.loc[i, 'id'] = df3.loc[i, 'alternate_url']
    matches.loc[i, 'description'] = df3.loc[i, 'description_stem']
    
    if i%1000==0:
        print(i, datetime.now())

0 2022-08-21 11:32:21.716259
1000 2022-08-21 11:32:24.439167
2000 2022-08-21 11:32:27.084831
3000 2022-08-21 11:32:29.819857
4000 2022-08-21 11:32:32.759811
5000 2022-08-21 11:32:35.810385
6000 2022-08-21 11:32:38.984302
7000 2022-08-21 11:32:42.568809
8000 2022-08-21 11:32:47.756570
9000 2022-08-21 11:32:52.939737
10000 2022-08-21 11:32:57.896838
11000 2022-08-21 11:33:04.342741
12000 2022-08-21 11:33:12.315536
13000 2022-08-21 11:33:19.630603
14000 2022-08-21 11:33:26.133704
15000 2022-08-21 11:33:32.594245
16000 2022-08-21 11:33:39.905781
17000 2022-08-21 11:33:48.404020
18000 2022-08-21 11:33:55.918497
19000 2022-08-21 11:34:03.227421
20000 2022-08-21 11:34:11.714625


In [56]:
matches.sort_values('Similarity', ascending = False)[:5]

,Similarity,id,description
3243,50.63,https://hh.ru/vacancy/68285003,в строительн компан профтехстр треб геодезист ...
12424,48.95,https://hh.ru/vacancy/68606551,наш компан явля высокотехнологичн компан в сфе...
15394,47.28,https://hh.ru/vacancy/68296252,ваканс предполага переезд в болив и работ на п...
13401,45.94,https://hh.ru/vacancy/68147056,ваканс компан оо стро тел сет оо стро тел сет ...
8757,45.48,https://hh.ru/vacancy/68681214,функциональн обязан график производств работ а...


In [64]:
matches = pd.DataFrame()
tfidf = TfidfVectorizer()

for i in range(len(df3)):
    
    resume = resumes2.loc[6, 'exp_desc_stem']
    vacancy = df3.loc[i, 'description_stem']
    
    Match_Test=[resume, vacancy]
    
    count_matrix=tfidf.fit_transform(Match_Test)
    
    MatchPercentage=cosine_similarity(count_matrix)[0][1]*100
    MatchPercentage=round(MatchPercentage,2)
    
    matches.loc[i, 'Similarity'] = round(MatchPercentage,2)
    matches.loc[i, 'id'] = df3.loc[i, 'alternate_url']
    matches.loc[i, 'description'] = df3.loc[i, 'description_stem']
    
    if i%1000==0:
        print(i, datetime.now())

0 2022-08-25 10:34:55.185036
1000 2022-08-25 10:34:59.971829
2000 2022-08-25 10:35:04.069277
3000 2022-08-25 10:35:07.820822
4000 2022-08-25 10:35:11.875225
5000 2022-08-25 10:35:16.369475
6000 2022-08-25 10:35:22.081971
7000 2022-08-25 10:35:31.752458
8000 2022-08-25 10:35:38.000795
9000 2022-08-25 10:35:43.550282
10000 2022-08-25 10:35:50.791924
11000 2022-08-25 10:35:57.123838
12000 2022-08-25 10:36:05.342722
13000 2022-08-25 10:36:12.074213
14000 2022-08-25 10:36:20.114804
15000 2022-08-25 10:36:27.386742
16000 2022-08-25 10:36:34.539297
17000 2022-08-25 10:36:41.360764
18000 2022-08-25 10:36:49.501613
19000 2022-08-25 10:36:57.542378
20000 2022-08-25 10:37:06.335839


In [65]:
matches.sort_values('Similarity', ascending = False)[:5]

,Similarity,id,description
12482,20.29,https://hh.ru/vacancy/68451584,эт перв в росс подписк на репетитор по всем шк...
20141,19.78,https://hh.ru/vacancy/66536862,обязан учет сопровожден и контрол процесс связ...
17368,18.95,https://hh.ru/vacancy/68185742,используем технологическ стек что нужн дела ак...
6156,18.38,https://hh.ru/vacancy/68685601,наш команд занима анализ дан и поиск точек рос...
16099,18.19,https://hh.ru/vacancy/68883784,мы ищ активн и инициативн аналитик для усилен ...


In [46]:
tfidf = TfidfVectorizer()

resume = resumes2.loc[1, 'exp_desc_stem']
vacancy = df3.loc[i, 'description_stem']
    
jobs = tfidf.fit_transform(df3["description_stem"].fillna(' '))
cv = tfidf.transform([resume])
    
MatchPercentage=list(cosine_similarity(jobs, cv).flatten())

In [47]:
matches = pd.DataFrame()

for i in range(len(df3)):
    matches.loc[i, 'Similarity'] = round(MatchPercentage[i]*100,2)
    matches.loc[i, 'id'] = df3.loc[i, 'alternate_url']
    matches.loc[i, 'description'] = df3.loc[i, 'description_stem']

In [48]:
matches.sort_values('Similarity', ascending = False)[:5]

,Similarity,id,description
13401,25.75,https://hh.ru/vacancy/68147056,ваканс компан оо стро тел сет оо стро тел сет ...
12424,24.94,https://hh.ru/vacancy/68606551,наш компан явля высокотехнологичн компан в сфе...
4621,24.22,https://hh.ru/vacancy/68441617,треб монтажник связ лвс скс волс скуд услов ра...
12463,23.74,https://hh.ru/vacancy/37410496,обязан монтаж плат и блок рэ требован профильн...
11338,23.58,https://hh.ru/vacancy/68122523,монтажник технологическ трубопровод разряд обя...
